In [1]:
%matplotlib inline
from PIL import Image
import matplotlib.pyplot as plt
import torch
import torchvision.transforms as T
import torchvision
import numpy as np

import cv2
import random
# import warnings
# warnings.filterwarnings('ignore')

In [2]:
# load model
model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained = True)
# set to evaluation mode
model.eval()

# load coco categories
COCO_CLASS_NAMES = [
    '__background__', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A', 'stop sign',
    'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
    'elephant', 'bear', 'zebra', 'giraffe', 'N/A', 'backpack', 'umbrella', 'N/A', 'N/A',
    'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
    'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket',
    'bottle', 'N/A', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl',
    'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
    'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'N/A', 'dining table',
    'N/A', 'N/A', 'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
    'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'N/A', 'book',
    'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'
]

Downloading: "https://download.pytorch.org/models/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth" to /Users/daniel/.cache/torch/hub/checkpoints/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth
100%|██████████| 170M/170M [02:54<00:00, 1.02MB/s]   


In [81]:
def get_coloured_mask(mask):
    # 10 colours
    colours = [[0, 255, 0],[0, 0, 255],[255, 0, 0],[0, 255, 255],[255, 255, 0],[255, 0, 255],[80, 70, 180],[250, 80, 190],[245, 145, 50],[70, 150, 250],[50, 190, 190]]
    r = np.zeros_like(mask).astype(np.uint8)
    g = np.zeros_like(mask).astype(np.uint8)
    b = np.zeros_like(mask).astype(np.uint8)
    r[mask == 1], g[mask == 1], b[mask == 1] = colours[random.randrange(0,10)]
    coloured_mask = np.stack([r, g, b], axis=2)
    return coloured_mask


def get_prediction(img_path, confidence):
    #img = Image.open(img_path)
    img = Image.fromarray(img_path)
    #print(type(img))
    
    transform = T.Compose([T.ToTensor()])
    img = transform(img)
    pred = model([img])
    pred_score = list(pred[0]['scores'].detach().numpy())
    pred_t = [pred_score.index(x) for x in pred_score if x>confidence][-1]
    masks = (pred[0]['masks']>0.5).squeeze().detach().cpu().numpy()
    # print(pred[0]['labels'].numpy().max())
    pred_class = [COCO_CLASS_NAMES[i] for i in list(pred[0]['labels'].numpy())]
    pred_boxes = [[(i[0], i[1]), (i[2], i[3])] for i in list(pred[0]['boxes'].detach().numpy())]
    masks = masks[:pred_t+1]
    pred_boxes = pred_boxes[:pred_t+1]
    pred_class = pred_class[:pred_t+1]
    return masks, pred_boxes, pred_class


def segment_instance(img_path, confidence = 0.5, rect_th = 2, text_size = 1.5, text_th = 2):
    masks, boxes, pred_cls = get_prediction(img_path, confidence)
    #####   #####   #####   #####   #####
    # img = cv2.imread(img_path)
    img = img_path
    #####   #####   #####   #####   #####


    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    for i in range(len(masks)):
        rgb_mask = get_coloured_mask(masks[i])
        img = cv2.addWeighted(img, 1, rgb_mask, 0.5, 0)
        cv2.rectangle(img, 
                        (int(boxes[i][0][0]), int(boxes[i][0][1])), 
                        (int(boxes[i][1][0]), int(boxes[i][1][1])), 
                        color = (0, 255, 0), 
                        thickness = rect_th)
        # cv2.rectangle(img, boxes[i][0], boxes[i][1], color = (0, 255, 0), thickness = rect_th)
        cv2.putText(img, 
                    pred_cls[i], 
                    (int(boxes[i][0][0]), int(boxes[i][0][1])),
                    # boxes[i][0], 
                    cv2.FONT_HERSHEY_SIMPLEX, 
                    text_size, 
                    (0,255,0), 
                    thickness = text_th)
    #plt.figure(figsize=(20,30))
    #plt.imshow(img)
    #plt.xticks([])
    #plt.yticks([])
    #plt.show()
    return img

# segment_instance('./traffic.jpg', confidence=0.7)
# segment_instance('./examples/prova.jpg', confidence=0.7)



In [82]:
vidcap = cv2.VideoCapture('./examples/FSD.mp4')
#success, image = vidcap.read()
# print(vidcap.get(cv2.CAP_PROP_FPS)) # fps

success = True
count = 0
frames_class = []

from tqdm import tqdm
for ele in tqdm(range(int(vidcap.get(7)))):
    if success:
        success, image = vidcap.read()
        frames_class.append(segment_instance(image, confidence=0.7))
        count += 1
    else:
        print(f'something went wrong in {count}')

100%|██████████| 2089/2089 [1:57:54<00:00,  3.39s/it] 


In [86]:
import pickle

with open("test_d", "wb") as fp:   #Pickling
    pickle.dump(frames_class, fp)

#>>> with open("test", "rb") as fp:   # Unpickling
#...   b = pickle.load(fp)

In [93]:
height, width, layers = frames_class[0].shape

final_video = cv2.VideoWriter('final.avi', 0, 30, (width, height))

for image in tqdm(frames_class):
    final_video.write(image)

cv2.destroyAllWindows()
final_video.release()

100%|██████████| 2089/2089 [00:09<00:00, 228.15it/s]
